In [1]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(PROJECT_ROOT)

from trial_class import *
from experiment_class import Experiment

import numpy as np
import pandas as pd
import tdt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from scipy.signal import butter, filtfilt
from sklearn.linear_model import LinearRegression
from sp_extension import *

In [2]:
experiment_path = r"C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Social_Pref\nac"
csv_base_path = r"C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Social_Pref\nac_csvs"

# NAc: #15616F
# mPFC: #FFAF00

cups = r"C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Social_Pref\Social_Pref_sheet.csv"


In [3]:
# groups csv + experiment data into one variable
experiment = Experiment(experiment_path, csv_base_path)

# batch process the data, removing the specified time segments for subjects
experiment.default_batch_process()

Found Synapse note file: C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Social_Pref\nac\n1-240522-072114\Notes.txt
read from t=0s to t=794.67s
Found Synapse note file: C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Social_Pref\nac\n2-240522-084131\Notes.txt
read from t=0s to t=789.95s
Found Synapse note file: C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Social_Pref\nac\n3-240523-073132\Notes.txt
read from t=0s to t=788.57s
Found Synapse note file: C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Social_Pref\nac\n4-240523-084829\Notes.txt
read from t=0s to t=790.88s
Found Synapse note file: C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Social_Pref\nac\n5-240826-083822\Notes.txt
read from t=0s to t=793.05s
Found Synapse note file: C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Social_Pref\nac\n6-240826-094701\Notes.txt
re

In [4]:
bout_definitions = [
    {'prefix': 'Subject', 'introduced': 'Subject Introduced', 'removed': 'Subject Removed'},
]


experiment.group_extract_manual_annotations(bout_definitions,first_only=True)

Processing behaviors for n1-240522-072114...
Processing behaviors for n2-240522-084131...
Processing behaviors for n3-240523-073132...
Processing behaviors for n4-240523-084829...
Processing behaviors for n5-240826-083822...
Processing behaviors for n6-240826-094701...
Processing behaviors for n7-240827-072608...


## Dopamine Stuff (Need help with this)

In [5]:
def prep_combined_da_metrics(experiment, sniff_cup_csv_path, metric_list=None, first_only=False):
    import pandas as pd
    import re

    # Normalize behavior label spacing
    def normalize_behavior_label(label):
        return re.sub(r'\s+', ' ', label.strip().lower().replace('\u00a0', ' '))

    assign_df = pd.read_csv(sniff_cup_csv_path)
    assign_df['Subject'] = assign_df['Subject'].astype(str).str.lower()

    # Build subject -> behavior name -> agent identity mapping
    subject_to_behavior_to_agent = {}
    for _, row in assign_df.iterrows():
        subj = row['Subject']
        subject_to_behavior_to_agent[subj] = {}
        for col in row.index:
            col_norm = normalize_behavior_label(str(col))
            if col_norm.startswith("sniff cup"):
                agent_label = normalize_behavior_label(str(row[col]))
                subject_to_behavior_to_agent[subj][col_norm] = agent_label

    all_rows = []

    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, 'behaviors') or trial.behaviors.empty:
            continue

        df = trial.behaviors.copy()
        df['Behavior'] = df['Behavior'].astype(str).apply(normalize_behavior_label)

        subject_id = trial_name.lower()

        if subject_id not in subject_to_behavior_to_agent:
            continue

        mapping = subject_to_behavior_to_agent[subject_id]

        # Keep only sniff cup behaviors
        df = df[df["Behavior"].str.startswith("sniff cup")]

        # Map behaviors to agents
        df["Agent"] = df["Behavior"].apply(lambda b: mapping.get(b))
        df["Subject"] = subject_id
        df["Trial"] = trial_name

        unmatched = df[df["Agent"].isna()]
        if not unmatched.empty:
            print(f"‼️ Unmatched behaviors for subject '{subject_id}':")
            print("Behaviors that failed to map:", unmatched["Behavior"].unique())
            print("Available mapping keys:", list(mapping.keys()))

        df = df.dropna(subset=["Agent"])

        # Choose metrics
        known_cols = ["Behavior", "Agent", "Subject", "Trial"]
        if metric_list:
            metric_cols = [m for m in metric_list if m in df.columns]
        else:
            metric_cols = [c for c in df.columns if c not in known_cols and pd.api.types.is_numeric_dtype(df[c])]

        if not metric_cols:
            continue

        df = df[["Subject", "Agent"] + metric_cols]

        if first_only:
            df = df.groupby(["Subject", "Agent"], as_index=False).first()

        all_rows.append(df)

    if not all_rows:
        print("⚠️ No rows added to DataFrame. Check if behavior labels match and mapping keys are clean.")
        print(f"Subjects in experiment: {list(experiment.trials.keys())}")
        print(f"Subjects in assignments file: {assign_df['Subject'].tolist()}")
        print("Sample mapping dictionary:")
        for subj, mapping in subject_to_behavior_to_agent.items():
            print(f"{subj} -> {mapping}")
        return pd.DataFrame()

    combined_df = pd.concat(all_rows, ignore_index=True)

    # --- Aggregate by Subject-Agent pair ---
    if first_only:
        grouped = combined_df  # already one row per subject-agent
    else:
        grouped = combined_df.groupby(["Subject", "Agent"], as_index=False)[metric_cols].mean()

    # --- Ensure each subject has all 4 agent rows ---
    all_agents = ['nothing', 'short_term', 'long_term', 'novel']
    all_subjects = sorted(grouped['Subject'].unique())
    full_index = pd.MultiIndex.from_product([all_subjects, all_agents], names=['Subject', 'Agent'])

    final_df = (
        grouped.set_index(['Subject', 'Agent'])
               .reindex(full_index)
               .fillna(0)
               .reset_index()
    )

    print(f"✅ Final DA metrics DataFrame created with {len(final_df)} rows from {len(all_subjects)} subjects.")
    return final_df



In [6]:
experiment.compute_all_da_metrics(use_max_length=False,
                                  max_bout_duration=5, #total_avg_bout_duration
                                  mode='standard')

Computing DA metrics for n1-240522-072114 ...
Computing DA metrics for n2-240522-084131 ...
Computing DA metrics for n3-240523-073132 ...
Computing DA metrics for n4-240523-084829 ...
Computing DA metrics for n5-240826-083822 ...
Computing DA metrics for n6-240826-094701 ...
Computing DA metrics for n7-240827-072608 ...


In [11]:
experiment

In [7]:

# Build the combined dataframe, aggregating by mean across trials
combined_df = prep_combined_da_metrics(
    experiment=experiment,
    sniff_cup_csv_path=cups,
    metric_list=["Max Peak", "Mean Z-score", "AUC"],
    first_only=True
)

combined_df

⚠️ No rows added to DataFrame. Check if behavior labels match and mapping keys are clean.
Subjects in experiment: ['n1-240522-072114', 'n2-240522-084131', 'n3-240523-073132', 'n4-240523-084829', 'n5-240826-083822', 'n6-240826-094701', 'n7-240827-072608']
Subjects in assignments file: ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
Sample mapping dictionary:
n1 -> {}
n2 -> {}
n3 -> {}
n4 -> {}
n5 -> {}
n6 -> {}
n7 -> {}
p1 -> {}
p2 -> {}
p3 -> {}
p4 -> {}
p5 -> {}
p6 -> {}
p7 -> {}
p8 -> {}


""


In [8]:
nac_pref_list = ['n5-240826-083822']
mpfc_pref_list =['p5-240826-091418', 'pp3-250118-064713', 'p7-240826-102402', 'p4-240523-092600']

nac_no_pref_list = ['nn2-250117-085631', 'nn4-250118-094351', 'n6-240826-094701', 'nn1-250117-081652', 'nn8-250118-105443', 'nn7-250118-101917', 'pp8-250118-083250', 'nn6-250117-101903', 'n7-240827-072608', 'nn3-250118-090940', 'nn5-250117-093631']
mpfc_no_pref_list =['p2-240523-081105','pp7-250118-075659', 'p8-240827-075823', 'p1-240522-080200', 'p6-240827-065303', 'pp5-250117-121543', 'pp4-250118-072201', 'pp6-250117-124823', 'p3-240522-092431','pp1-250117-110456', 'pp2-250117-113909']

nac_list=['nn2-250117-085631', 'nn4-250118-094351', 'n6-240826-094701', 'nn1-250117-081652', 'nn8-250118-105443', 'nn7-250118-101917', 'pp8-250118-083250', 'nn6-250117-101903', 'n7-240827-072608', 'nn3-250118-090940', 'nn5-250117-093631', 'n5-240826-083822']
mpfc_list=['p5-240826-091418', 'pp3-250118-064713', 'p7-240826-102402', 'p4-240523-092600','p2-240523-081105','pp7-250118-075659', 'p8-240827-075823', 'p1-240522-080200', 'p6-240827-065303', 'pp5-250117-121543', 'pp4-250118-072201', 'pp6-250117-124823', 'p3-240522-092431','pp1-250117-110456', 'pp2-250117-113909']

In [9]:
def dopamine(precomputed_df, 
             metric_name="Mean Z-score", 
             title="Combined DA Metrics", 
             ylabel="DA Metric", 
             xlabel="Agent", 
             custom_xtick_labels=None, 
             custom_xtick_colors=None, 
             ylim=None, 
             bar_color="#00B7D7", 
             yticks_increment=None, 
             figsize=(14, 8), 
             pad_inches=0.1,
             save=False,
             save_name=None,
             subjects_to_include=None,
             highlight_subject=None):
    """
    Plots DA metrics across agents ("nothing", "short_term", "long_term", "novel")
    with subject-level spaghetti plots and paired t-tests.
    """

    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from scipy.stats import ttest_rel

    fixed_order = ["nothing", "short_term", "long_term", "novel"]
    bar_positions = np.arange(len(fixed_order))

    def perform_all_pairwise_t_tests(pivot_df):
        results = {}
        bout_names = pivot_df.columns.tolist()
        for i in range(len(bout_names)):
            for j in range(i + 1, len(bout_names)):
                bout1, bout2 = bout_names[i], bout_names[j]
                paired_df = pivot_df[[bout1, bout2]].dropna()
                if len(paired_df) > 1:
                    t_stat, p_value = ttest_rel(paired_df[bout1], paired_df[bout2])
                    results[f"{bout1} vs {bout2}"] = {"t_stat": t_stat, "p_value": p_value}
        return results

    df = precomputed_df.copy()

    # --- Filter by Subject ---
    if subjects_to_include is not None:
        subjects_to_include = [s.lower() for s in subjects_to_include]
        df['Subject'] = df['Subject'].astype(str).str.lower()
        df = df[df['Subject'].isin(subjects_to_include)]

    if df.empty:
        print("⚠️ No data to plot after filtering.")
        return

    # --- Pivot by Subject x Agent ---
    try:
        pivot_df = df.pivot(index="Subject", columns="Agent", values=metric_name)
    except Exception as e:
        print("Error pivoting data:", e)
        return

    pivot_df = pivot_df.reindex(columns=fixed_order)

    # --- Summary Stats ---
    stats = pivot_df.agg(['mean', 'sem']).T.reset_index()
    stats.columns = ['Agent', 'mean', 'sem']
    stats = stats.set_index('Agent').reindex(fixed_order).reset_index()

    means = stats['mean'].values
    sems = stats['sem'].values

    # --- Paired T-tests ---
    t_test_results = perform_all_pairwise_t_tests(pivot_df)

    # --- Plot ---
    fig, ax = plt.subplots(figsize=figsize)

    # Bars
    ax.bar(
        bar_positions,
        means,
        yerr=sems,
        capsize=10,
        color=bar_color,
        edgecolor='black',
        linewidth=5,
        width=0.6
    )

    # Spaghetti lines (gray for all, black for highlight)
    for subject_id, row in pivot_df.iterrows():
        if highlight_subject and subject_id.lower() == highlight_subject.lower():
            ax.plot(bar_positions, row.values, linestyle='-', color='black', linewidth=4, zorder=3)
            ax.scatter(bar_positions, row.values, facecolors='black', edgecolors='black', 
                       s=160, linewidths=2, zorder=4)
        else:
            ax.plot(bar_positions, row.values, linestyle='-', color='gray', alpha=0.5, linewidth=2.5, zorder=1)
            ax.scatter(bar_positions, row.values, facecolors='none', edgecolors='gray', 
                       s=120, linewidths=3, zorder=2)

    # Labels
    ax.set_ylabel(ylabel, fontsize=35, labelpad=12)
    ax.set_xlabel(xlabel, fontsize=35, labelpad=12)
    if title:
        ax.set_title(title, fontsize=28)

    ax.set_xticks(bar_positions)
    xtick_labels = custom_xtick_labels if custom_xtick_labels else ["Empty", "Short Term", "Long Term", "Novel"]
    ax.set_xticklabels(xtick_labels, fontsize=35)
    if custom_xtick_colors:
        for tick, color in zip(ax.get_xticklabels(), custom_xtick_colors):
            tick.set_color(color)

    ax.tick_params(axis='y', labelsize=35)
    ax.tick_params(axis='x', labelsize=35)

    # Y-limits
    if ylim:
        ax.set_ylim(ylim)
    else:
        all_vals = np.concatenate([pivot_df.values.flatten(), means])
        ax.set_ylim(0, np.nanmax(all_vals) * 1.2)

    if yticks_increment:
        y_min, y_max = ax.get_ylim()
        ax.set_yticks(np.arange(np.floor(y_min), np.ceil(y_max) + yticks_increment, yticks_increment))

    ax.axhline(y=0, color='black', linestyle='--', linewidth=2)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(5)
    ax.spines['bottom'].set_linewidth(5)

    plt.tight_layout()
    if save:
        if save_name is None:
            raise ValueError("save_name must be provided if save is True.")
        plt.savefig(save_name, transparent=True, bbox_inches='tight', pad_inches=pad_inches)
    plt.show()

    # --- Print T-tests ---
    print(f"\nPlotted data from {pivot_df.shape[0]} subject(s).")
    if t_test_results:
        print("\nPaired t-test results (all agent comparisons):")
        for comp, stats in t_test_results.items():
            p = stats["p_value"]
            stars = "ns"
            if p < 0.001:
                stars = "***"
            elif p < 0.01:
                stars = "**"
            elif p < 0.05:
                stars = "*"
            print(f"{comp}: p = {p:.4f} ({stars})")

labels=["Empty", "Short Term", "Long Term", "Novel"]
colors=["black", "blue", "purple", "orange"]

dopamine(
    precomputed_df=combined_df,
    metric_name="Max Peak",
    ylabel="Standard Peak ∆F/F",
    xlabel="Agent",
    custom_xtick_labels=labels,
    custom_xtick_colors=colors,
    bar_color="#FFAF00",
    ylim=(-2, 4),
    yticks_increment=2,
    figsize=(14, 8),
    save=True,
    save_name ="mpfc_pref_DA",
    title=None
)
dopamine(
    precomputed_df=combined_df,
    metric_name="Max Peak",
    ylabel="Standard Peak ∆F/F",
    xlabel="Agent",
    custom_xtick_labels=labels,
    custom_xtick_colors=colors,
    bar_color="#FFAF00",
    ylim=(-2, 4),
    yticks_increment=2,
    figsize=(14, 8),
    save=True,
    save_name ="mpfc_no_pref_DA",
    title=None
)

dopamine(
    precomputed_df=combined_df,
    metric_name="Max Peak",
    ylabel="Standard Peak ∆F/F",
    xlabel="Agent",
    custom_xtick_labels=labels,
    custom_xtick_colors=colors,
    bar_color="#15616F",
    ylim=(-2, 10),
    yticks_increment=2,
    figsize=(14, 8),
    save=True,
    save_name ="nac_pref_DA",
    title=None
)

dopamine(
    precomputed_df=combined_df,
    metric_name="Max Peak",
    ylabel="Standard Peak ∆F/F",
    xlabel="Agent",
    custom_xtick_labels=labels,
    custom_xtick_colors=colors,
    bar_color="#15616F",
    ylim=(-2, 10),
    yticks_increment=2,
    figsize=(14, 8),
    save=True,
    save_name ="nac_no_pref_DA",
    title=None
)

⚠️ No data to plot after filtering.
⚠️ No data to plot after filtering.
⚠️ No data to plot after filtering.
⚠️ No data to plot after filtering.


## DA across subsequent bouts

In [10]:
# 1. Re-extract all behaviors and re-calculate DA metrics
experiment.reset_all_behaviors()
experiment.group_extract_manual_annotations(bout_definitions=bout_definitions, first_only=False)

experiment.compute_all_da_metrics(
    use_max_length=False,
    max_bout_duration=4,
    use_adaptive=False,
    allow_bout_extension=False,
    mode='standard'
)

Processing behaviors for n1-240522-072114...
Processing behaviors for n2-240522-084131...
Processing behaviors for n3-240523-073132...
Processing behaviors for n4-240523-084829...
Processing behaviors for n5-240826-083822...
Processing behaviors for n6-240826-094701...
Processing behaviors for n7-240827-072608...


TypeError: Experiment.compute_all_da_metrics() got an unexpected keyword argument 'use_adaptive'

In [ ]:
def get_all_per_event_df(experiment, sniff_cup_csv_path, metric_list=None):
    import re

    def normalize_label(label):
        return re.sub(r'\s+', ' ', str(label).strip().lower().replace('\u00a0', ' '))

    # Load mapping from cup -> agent
    assign_df = pd.read_csv(sniff_cup_csv_path)
    assign_df['Subject'] = assign_df['Subject'].str.lower()
    
    subj_map = {}
    for _, row in assign_df.iterrows():
        subj = row['Subject']
        subj_map[subj] = {}
        for col in row.index:
            col_norm = normalize_label(col)
            if col_norm.startswith("sniff cup"):
                agent = normalize_label(row[col])
                subj_map[subj][col_norm] = agent

    all_rows = []
    for trial_name, trial in experiment.trials.items():
        if not hasattr(trial, 'behaviors') or trial.behaviors.empty:
            continue

        df = trial.behaviors.copy()
        df['Behavior'] = df['Behavior'].astype(str).apply(normalize_label)

        subject_id = trial_name.lower()
        if subject_id not in subj_map:
            continue

        df = df[df['Behavior'].str.startswith("sniff cup")]
        df['Agent'] = df['Behavior'].apply(lambda b: subj_map[subject_id].get(b))
        df['Subject'] = subject_id
        df['Trial'] = trial_name

        df = df.dropna(subset=["Agent"])
        
        known = ['Subject', 'Agent', 'Trial', 'Behavior', 'Event_Start']
        if metric_list:
            cols = [col for col in metric_list if col in df.columns]
        else:
            cols = [c for c in df.columns if c not in known and pd.api.types.is_numeric_dtype(df[c])]

        df = df[['Subject', 'Agent', 'Trial', 'Event_Start'] + cols]
        all_rows.append(df)

    final_df = pd.concat(all_rows, ignore_index=True)
    print(f"✅ Per-event DataFrame created with {len(final_df)} rows from {final_df['Subject'].nunique()} subjects.")
    return final_df

per_event_df = get_all_per_event_df(experiment, sniff_cup_csv_path=cups, metric_list=["Max Peak", "Mean Z-score", "AUC"])

In [ ]:
def plot_peak_by_agent_from_df(
    df,
    sniff_cup_csv_path=None,              # optional if Agent column is already present
    selected_agents=None,                # e.g. ['novel', 'short_term']
    n_subsequent_investigations=3,
    peak_col="Max Peak",
    metric_type='slope',
    figsize=(14, 8),
    line_order=None,
    custom_colors=None,
    custom_legend_labels=None,
    custom_xtick_labels=None,
    ylim=None,
    ytick_increment=None,
    xlabel="Investigation Index",
    ylabel="Avg Max Peak",
    subjects_to_include=None,            # ✅ MISSING COMMA FIXED HERE
    plot_title="Average Peak per Agent",
    save=False,
    save_name="agent_peak_plot.png"
):
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.stats import linregress
    from scipy.optimize import curve_fit

    def exponential_decay(x, A, B, tau):
        return A + B * np.exp(-x / tau)

    def normalize_label(label):
        import re
        return re.sub(r'\s+', ' ', str(label).strip().lower().replace('\u00a0', ' '))

    def create_mapping(sniff_cup_csv_path):
        assign_df = pd.read_csv(sniff_cup_csv_path)
        assign_df['Subject'] = assign_df['Subject'].astype(str).str.lower()
        subject_to_behavior_to_agent = {}
        for _, row in assign_df.iterrows():
            subj = row['Subject']
            subject_to_behavior_to_agent[subj] = {}
            for col in row.index:
                col_norm = normalize_label(col)
                if col_norm.startswith("sniff cup"):
                    agent_label = normalize_label(row[col])
                    subject_to_behavior_to_agent[subj][col_norm] = agent_label
        return subject_to_behavior_to_agent

    df = df.copy()

    # --- Optional agent mapping ---
    if "Agent" not in df.columns:
        if sniff_cup_csv_path is None:
            raise ValueError("You must provide either an 'Agent' column or a sniff_cup_csv_path.")

        mapping = create_mapping(sniff_cup_csv_path)

        def get_agent(row):
            subj = str(row['Subject']).lower()
            bout = str(row['Bout']).lower()
            if '-' not in bout:
                return None
            cup_number = bout.split('-')[1]
            behavior = f"sniff cup {cup_number}"
            return mapping.get(subj, {}).get(behavior)

        df['Agent'] = df.apply(get_agent, axis=1)

    # --- Filter agents ---
    if selected_agents:
        df = df[df['Agent'].isin(selected_agents)]

    # --- Subject filtering ---
    if subjects_to_include:
        subjects_to_include = [s.lower() for s in subjects_to_include]
        df['Subject'] = df['Subject'].astype(str).str.lower()
        df = df[df['Subject'].isin(subjects_to_include)]

    # --- Investigation indexing ---
    df.sort_values(["Subject", "Agent", "Event_Start"], inplace=True)
    df["InvestigationIndex"] = df.groupby(["Subject", "Agent"]).cumcount() + 1
    df = df[df["InvestigationIndex"] <= n_subsequent_investigations]

    # --- Aggregate ---
    agg_df = (
        df.groupby(["Agent", "InvestigationIndex"], as_index=False)
        .agg(
            SubjectCount=("Subject", "nunique"),
            AvgPeak=(peak_col, "mean")
        )
    )

    # --- Plotting ---
    if custom_colors is None:
        custom_colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

    fig, ax = plt.subplots(figsize=figsize)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_linewidth(5)
    ax.spines["bottom"].set_linewidth(5)
    ax.tick_params(axis="both", which="major", labelsize=48)
    ax.tick_params(axis="y", labelsize=35)

    metrics_dict = {}
    unique_agents = line_order if line_order else sorted(agg_df["Agent"].dropna().unique())

    for i, agent in enumerate(unique_agents):
        df_line = agg_df[agg_df["Agent"] == agent].copy()
        df_line.sort_values("InvestigationIndex", inplace=True)

        x_vals = df_line["InvestigationIndex"].values
        y_vals = df_line["AvgPeak"].values

        if len(x_vals) == 0 or len(y_vals) == 0:
            print(f"Skipping agent '{agent}' due to no data.")
            continue

        if metric_type.lower() == 'slope':
            slope, _, _, _, _ = linregress(x_vals, y_vals)
            metrics_dict[agent] = slope
            metric_label = f"slope: {slope:.3f}"
        elif metric_type.lower() == 'decay':
            try:
                p0 = (np.min(y_vals), np.max(y_vals)-np.min(y_vals), 1.0)
                popt, _ = curve_fit(exponential_decay, x_vals, y_vals, p0=p0)
                tau = popt[2]
                metrics_dict[agent] = tau
                metric_label = f"decay: {tau:.3f}"
            except RuntimeError:
                metrics_dict[agent] = np.nan
                metric_label = "decay: N/A"
        else:
            raise ValueError("metric_type must be 'slope' or 'decay'.")

        legend_label = custom_legend_labels[i] if custom_legend_labels and i < len(custom_legend_labels) else agent
        legend_label += f" ({metric_label}, n={df_line['SubjectCount'].max()})"

        color = custom_colors[i % len(custom_colors)]
        ax.plot(
            x_vals, y_vals,
            marker='o', linestyle='-',
            color=color,
            linewidth=5, markersize=30,
            label=legend_label
        )

    ax.set_xlabel(xlabel, fontsize=35, labelpad=12)
    ax.set_ylabel(ylabel, fontsize=35, labelpad=12)

    if ylim is not None:
        ax.set_ylim(ylim)
        if ytick_increment is not None:
            ticks = np.arange(ylim[0], ylim[1] + ytick_increment, ytick_increment)
            ax.set_yticks(ticks)
            ax.set_yticklabels([f"{t:.0f}" if t.is_integer() else f"{t:.1f}" for t in ticks], fontsize=35)

    if custom_xtick_labels:
        ax.set_xticks(np.arange(1, len(custom_xtick_labels) + 1))
        ax.set_xticklabels(custom_xtick_labels, fontsize=35)
    else:
        x_vals = sorted(agg_df["InvestigationIndex"].unique())
        ax.set_xticks(x_vals)
        ax.set_xticklabels([str(x) for x in x_vals], fontsize=35)

    if plot_title:
        ax.set_title(plot_title, fontsize=24)

    ax.legend(fontsize=26)
    plt.tight_layout()

    if save:
        plt.savefig(save_name, dpi=300, transparent=True, bbox_inches='tight')

    plt.show()

    print(f"\n=== Computed Metric ({metric_type.upper()}): ===")
    for agent, val in metrics_dict.items():
        print(f"Agent: {agent}, {metric_type} = {val:.3f}")

    return agg_df

lines = ['nothing','short_term','long_term','novel']

plot_peak_by_agent_from_df(
    df=per_event_df,
    selected_agents=lines,
    peak_col="Max Peak",
    metric_type="slope",  # or "slope"
    custom_colors = ['black','#0045A6','#A839A4','#E06928'],
    custom_legend_labels=['Empty', 'Short Term', 'Long Term', 'Novel'],
    line_order=lines,
    n_subsequent_investigations=5,
    ylim=(-1, 4),
    ytick_increment=2,
    ylabel = 'Average Peak ΔF/F',
    subjects_to_include=mpfc_pref_list,
    plot_title=None,
    save=False,
    save_name=""
)

plot_peak_by_agent_from_df(
    df=per_event_df,
    selected_agents=lines,
    peak_col="Max Peak",
    metric_type="slope",  # or "slope"
    custom_colors = ['black','#0045A6','#A839A4','#E06928'],
    custom_legend_labels=['Empty', 'Short Term', 'Long Term', 'Novel'],
    line_order=lines,
    n_subsequent_investigations=5,
    ylim=(-1, 3),
    ytick_increment=1,
    ylabel = 'Average Peak ΔF/F',
    subjects_to_include=mpfc_no_pref_list,
    plot_title=None,
    save=False,
    save_name=""
)

plot_peak_by_agent_from_df(
    df=per_event_df,
    selected_agents=lines,
    peak_col="Max Peak",
    metric_type="slope",  # or "slope"
    custom_colors = ['black','#0045A6','#A839A4','#E06928'],
    custom_legend_labels=['Empty', 'Short Term', 'Long Term', 'Novel'],
    line_order=lines,
    n_subsequent_investigations=5,
    ylim=(-4, 10),
    ytick_increment=2,
    ylabel = 'Average Peak ΔF/F',
    subjects_to_include=nac_pref_list,
    plot_title=None,
    save=False,
    save_name=""
)

plot_peak_by_agent_from_df(
    df=per_event_df,
    selected_agents=lines,
    peak_col="Max Peak",
    metric_type="slope",  # or "slope"
    custom_colors = ['black','#0045A6','#A839A4','#E06928'],
    custom_legend_labels=['Empty', 'Short Term', 'Long Term', 'Novel'],
    line_order=lines,
    n_subsequent_investigations=5,
    ylim=(-0.5, 3.5),
    ytick_increment=1,
    ylabel = 'Average Peak ΔF/F',
    subjects_to_include=nac_no_pref_list,
    plot_title=None,
    save=False,
    save_name=""
)

## DA vs % TIT

In [ ]:
# 1. Re-extract all behaviors and re-calculate DA metrics
experiment.reset_all_behaviors()
experiment.group_extract_manual_annotations(bout_definitions=bout_definitions, first_only=True)

In [ ]:
cups = "/Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/Social_Pref_Cup_Assignments.csv"

# Correct data input
trial_data_with_ids = get_trial_dataframes_with_ids(experiment)

# Create metadata with mapping
metadata_df = create_metadata_dataframe_with_agent_mapping(trial_data_with_ids, sniff_cup_csv_path=cups)

metadata_df

In [ ]:
def plot_da_vs_percent_investigation_time_by_agent(
    dopamine_df,
    metadata_df,
    agents_of_interest,
    agent_colors,
    agent_labels,
    da_metric='Max Peak',
    use_first_only=True,
    subjects_to_include=None,
    title="DA vs % Investigation Time",
    ylabel=None,
    figsize=(10, 7),
    ylim=None,
    yticks_increment=None,
    legend_loc='upper left',
    pad_inches=0.1,
    save=False,
    save_name=None
):
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.ticker as ticker
    from scipy.stats import linregress

    # --- Prep subject IDs ---
    if "Subject" not in metadata_df.columns:
        metadata_df = metadata_df.reset_index()
    metadata_df["Subject"] = metadata_df["Subject"].astype(str).str.lower()
    dopamine_df["Subject"] = dopamine_df["Subject"].astype(str).str.lower()

    if subjects_to_include:
        subjects_to_include = [s.lower() for s in subjects_to_include]
        metadata_df = metadata_df[metadata_df["Subject"].isin(subjects_to_include)]
        dopamine_df = dopamine_df[dopamine_df["Subject"].isin(subjects_to_include)]

    # --- Corrected: Calculate percent investigation time for selected agents ---
    percent_records = []

    # Get all agent names from metadata by parsing columns like "Total_novel"
    all_agent_names = [
        col.replace("Total_", "") for col in metadata_df.columns if col.startswith("Total_")
    ]

    for _, row in metadata_df.iterrows():
        subj = row["Subject"]

        # Total investigation time across all agents (not just selected ones)
        total_all = sum([
            row.get(f"Total_{agent}", 0)
            for agent in all_agent_names
        ])

        if total_all == 0:
            continue  # skip subject if no total investigation time at all

        for agent in agents_of_interest:
            agent_time = row.get(f"Total_{agent}", 0)
            percent = agent_time / total_all

            percent_records.append({
                "Subject": subj,
                "Agent": agent,
                "PercentTime": percent
            })

    percent_df = pd.DataFrame(percent_records)


    # --- Prepare DA values ---
    if use_first_only:
        dopamine_df = dopamine_df.groupby(["Subject", "Agent"], as_index=False).first()
    else:
        dopamine_df = dopamine_df.groupby(["Subject", "Agent"], as_index=False)[da_metric].mean()

    # --- Merge time + DA ---
    merged_df = pd.merge(dopamine_df, percent_df, on=["Subject", "Agent"], how="inner")

    if merged_df.empty:
        print("⚠️ No data to plot after merging.")
        return pd.DataFrame()

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=figsize)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_linewidth(3)
    ax.spines['bottom'].set_linewidth(3)

    all_x, all_y = [], []

    for agent in agents_of_interest:
        sub = merged_df[merged_df["Agent"] == agent]
        if sub.empty:
            continue

        x = sub["PercentTime"].values * 100
        y = sub[da_metric].values

        all_x.extend(x)
        all_y.extend(y)

        color = agent_colors.get(agent, 'gray')
        label = agent_labels.get(agent, agent)

        ax.scatter(x, y, color=color, s=250, alpha=1.0, edgecolor='black', linewidth=3, label=label, zorder=3)

    # --- Regression line ---
    stats_text_lines = ["r = ---", "p = ---", "n = ---"]
    if len(set(all_x)) > 1:  # ← Fix: only do regression if x-values aren't all identical
        slope, intercept, r_val, p_val, _ = linregress(all_x, all_y)
        x_fit = np.linspace(min(all_x), max(all_x), 100)
        y_fit = slope * x_fit + intercept
        ax.plot(x_fit, y_fit, color='black', linewidth=2.5, linestyle='-', zorder=2)

        stats_text_lines = [
            f"r = {r_val:.3f}",
            f"p = {p_val:.3f}",
            f"n = {len(all_x)} points"
        ]

    ax.set_xlabel("% Investigation Time", fontsize=24)
    ax.set_ylabel(ylabel if ylabel else da_metric, fontsize=24)
    ax.set_title(title, fontsize=26)
    ax.tick_params(axis='both', labelsize=24)

    if ylim:
        ax.set_ylim(ylim)
    if yticks_increment:
        y_min, y_max = ax.get_ylim()
        yticks = np.arange(np.floor(y_min), np.ceil(y_max) + yticks_increment, yticks_increment)
        ax.set_yticks(yticks)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x)}'))
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{int(x)}' if x.is_integer() else f'{x:.1f}'))

    handles, labels = ax.get_legend_handles_labels()
    stats_handle = plt.Line2D([], [], color='none', label="\n".join(stats_text_lines))
    handles.append(stats_handle)
    labels.append("\n".join(stats_text_lines))

    legend = ax.legend(handles=handles, labels=labels, loc=legend_loc, fontsize=16, title='Agent', title_fontsize=18,
                       frameon=True, facecolor='white', edgecolor='lightgray', fancybox=False)
    legend.get_frame().set_alpha(0.8)

    plt.tight_layout()
    if save:
        if save_name is None:
            raise ValueError("save_name must be provided if save is True.")
        plt.savefig(save_name, transparent=True, bbox_inches='tight', pad_inches=pad_inches)

    plt.show()

    return merged_df


plot_da_vs_percent_investigation_time_by_agent(
    dopamine_df=combined_df,  # your DA dataframe with 'Subject', 'Agent', DA metrics
    metadata_df=metadata_df,  # from your time analysis notebook
    agents_of_interest=["nothing"],
    agent_colors={"nothing": "black"},
    agent_labels={"novel": "Novel", "short_term": "Short Term", "long_term": "Long Term", "nothing": "Empty"},
    da_metric="Max Peak",
    use_first_only=True,
    title="NAc Nothing",
    ylabel="Peak Z-scored ∆F/F", 
    subjects_to_include=nac_list,
    legend_loc='upper right',
    save=False,
    save_name="da_vs_invest_time_corr.png"
)

plot_da_vs_percent_investigation_time_by_agent(
    dopamine_df=combined_df,  # your DA dataframe with 'Subject', 'Agent', DA metrics
    metadata_df=metadata_df,  # from your time analysis notebook
    agents_of_interest=["novel"],
    agent_colors={"novel": "#E06928"},
    agent_labels={"novel": "Novel"},
    da_metric="Max Peak",
    use_first_only=True,
    title="NAc Novel",
    ylabel="Peak Z-scored ∆F/F", 
    subjects_to_include=nac_list,
    legend_loc='upper right',
    save=False,
    save_name="da_vs_invest_time_corr.png"
)

plot_da_vs_percent_investigation_time_by_agent(
    dopamine_df=combined_df,  # your DA dataframe with 'Subject', 'Agent', DA metrics
    metadata_df=metadata_df,  # from your time analysis notebook
    agents_of_interest=["short_term"],
    agent_colors={"short_term": "#0045A6"},
    agent_labels={"short_term": "Short Term",},
    da_metric="Max Peak",
    use_first_only=True,
    title="NAc Short Term",
    ylabel="Peak Z-scored ∆F/F", 
    subjects_to_include=nac_list,
    legend_loc='upper right',
    save=False,
    save_name="da_vs_invest_time_corr.png"
)

plot_da_vs_percent_investigation_time_by_agent(
    dopamine_df=combined_df,  # your DA dataframe with 'Subject', 'Agent', DA metrics
    metadata_df=metadata_df,  # from your time analysis notebook
    agents_of_interest=["long_term"],
    agent_colors={"long_term": "#A839A4"},
    agent_labels={"long_term": "Long Term"},
    da_metric="Max Peak",
    use_first_only=True,
    title="NAc Long Term",
    ylabel="Peak Z-scored ∆F/F", 
    subjects_to_include=nac_list,
    legend_loc='lower right',
    save=False,
    save_name="da_vs_invest_time_corr.png"
)

## OLD

In [ ]:
raw_columns = [
    "Behavior", "Bout", "Event_Start", "Event_End", "Duration (s)", "AUC",
    "Max Peak", "Time of Max Peak", "Mean Z-score", "Adjusted End",
    "Relative_Time_Axis", "Relative_Zscore"
]

for trial in experiment.trials.values():
    if hasattr(trial, "behaviors") and not trial.behaviors.empty:
        df = trial.behaviors.copy()
        df.columns = raw_columns

        # Force all rows to be labeled as 'Investigation' for behavior
        df["Behavior"] = "Investigation"
        trial.behaviors = df

In [ ]:
exp_da_dict = get_trial_dataframes(experiment)

In [ ]:
desired_bouts = ['sniff cup 1', 'sniff cup 2', 'sniff cup 3', 'sniff cup 4']
da_metadata_df = create_da_metrics_dataframe(
    exp_da_dict,
    behavior="Investigation",
    desired_bouts=desired_bouts
)

da_metadata_df

,Subject,Bout,Behavior,AUC,Max Peak,Mean Z-score
0,p5,sniff cup 1,investigation,0.000000,0.000000,0.000000
1,p5,sniff cup 2,investigation,-11.359314,1.420445,-1.622571
2,p5,sniff cup 3,investigation,0.000000,0.000000,0.000000
3,p5,sniff cup 4,investigation,0.000000,0.000000,0.000000
4,nn2,sniff cup 1,investigation,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
87,pp1,sniff cup 4,investigation,0.000000,0.000000,0.000000
88,pp2,sniff cup 1,investigation,0.000000,0.000000,0.000000
89,pp2,sniff cup 2,investigation,0.000000,0.000000,0.000000
90,pp2,sniff cup 3,investigation,4.075799,1.629273,0.463285


In [ ]:
def map_sniff_cups_to_agents(da_df, cup_assignments_csv_path):
    """
    Maps sniff cup labels (e.g., 'sniff cup 1') to agent types 
    using subject-specific assignments from a CSV.
    """
    # Load and normalize the assignment CSV
    cup_df = pd.read_csv(cup_assignments_csv_path)
    cup_df["Subject"] = cup_df["Subject"].str.strip().str.lower()

    # Extract subject prefix (e.g., 'p5' from 'p5-240826-091418')
    cup_df["Subject_Prefix"] = cup_df["Subject"].str.split("-").str[0]

    # Melt long format
    mapping_long = cup_df.melt(
        id_vars=["Subject", "Subject_Prefix"],
        var_name="Cup_Label",
        value_name="Agent_Type"
    )
    mapping_long["Cup_Label"] = mapping_long["Cup_Label"].str.strip().str.lower()
    mapping_long["Agent_Type"] = mapping_long["Agent_Type"].str.strip().str.lower()

    # Normalize da_df
    da_df = da_df.copy()
    da_df["Subject_clean"] = da_df["Subject"].str.strip().str.lower()
    da_df["Bout_clean"] = da_df["Bout"].str.strip().str.lower()

    # Merge by prefix + cup label
    merged_df = da_df.merge(
        mapping_long,
        left_on=["Subject_clean", "Bout_clean"],
        right_on=["Subject_Prefix", "Cup_Label"],
        how="left"
    )

    # Clean up
    merged_df.drop(columns=["Subject_clean", "Bout_clean", "Cup_Label", "Subject_Prefix", "Subject_y"], inplace=True)
    merged_df.rename(columns={"Subject_x": "Subject"}, inplace=True)

    return merged_df


cups = "/Users/naylajimenez/Downloads/papers/dopamine/cohort-1-2/allcohorts/Cup_Assignments.csv"
da_with_agents_df = map_sniff_cups_to_agents(da_metadata_df, cups)
da_with_agents_df

,Subject,Bout,Behavior,AUC,Max Peak,Mean Z-score,Agent_Type
0,p5,sniff cup 1,investigation,0.000000,0.000000,0.000000,nothing
1,p5,sniff cup 2,investigation,-11.359314,1.420445,-1.622571,novel
2,p5,sniff cup 3,investigation,0.000000,0.000000,0.000000,short_term
3,p5,sniff cup 4,investigation,0.000000,0.000000,0.000000,long_term
4,nn2,sniff cup 1,investigation,0.000000,0.000000,0.000000,long_term
...,...,...,...,...,...,...,...
87,pp1,sniff cup 4,investigation,0.000000,0.000000,0.000000,long_term
88,pp2,sniff cup 1,investigation,0.000000,0.000000,0.000000,long_term
89,pp2,sniff cup 2,investigation,0.000000,0.000000,0.000000,nothing
90,pp2,sniff cup 3,investigation,4.075799,1.629273,0.463285,novel


In [ ]:
Pref = ['p5-240826-091418', 'pp3-250118-064713', 'pp5-250117-121543', 'pp4-250118-072201', 'p7-240826-102402', 'nn3-250118-090940', 'p4-240523-092600']
No_Pref = ['nn2-250117-085631', 'p2-240523-081105', 'nn4-250118-094351', 'nn1-250117-081652', 'pp7-250118-075659', 'p8-240827-075823', 'nn8-250118-105443', 'p1-240522-080200', 'p6-240827-065303', 'nn7-250118-101917', 'pp8-250118-083250', 'nn6-250117-101903', 'pp6-250117-124823', 'p3-240522-092431', 'nn5-250117-093631', 'pp1-250117-110456', 'pp2-250117-113909']

def plot_da_by_agent_type(precomputed_df, 
             metric_name="Mean Z-score", 
             title="DA Metrics by Agent Type", 
             ylabel="DA Metric", 
             xlabel="Agent Type", 
             custom_xtick_labels=None, 
             custom_xtick_colors=None, 
             ylim=None, 
             bar_color="#00B7D7", 
             yticks_increment=None, 
             figsize=(14,8), 
             pad_inches=0.1,
             save=False,
             save_name=None,
             subjects_to_include=None,
             agent_order=None):  # ✅ New param
    """
    Plots DA metrics across agent types using a provided DataFrame.
    Supports subject filtering and manual agent ordering.
    """

    from scipy.stats import ttest_rel
    import numpy as np
    import matplotlib.pyplot as plt

    def perform_t_tests(pivot_df):
        results = {}
        agent_types = pivot_df.columns.tolist()
        for i in range(1, len(agent_types)):
            agent1, agent2 = agent_types[0], agent_types[i]
            paired_df = pivot_df[[agent1, agent2]].dropna()
            if len(paired_df) > 1:
                t_stat, p_value = ttest_rel(paired_df[agent1], paired_df[agent2])
                significance = "ns"
                if p_value < 0.001:
                    significance = "***"
                elif p_value < 0.01:
                    significance = "**"
                elif p_value < 0.05:
                    significance = "*"
                results[f"{agent1} vs {agent2}"] = {
                    "t_stat": t_stat, 
                    "p_value": p_value, 
                    "significance": significance
                }
        return results

    df = precomputed_df.copy()

    # ✅ Filter by subject ID
    if subjects_to_include is not None:
        df = df[df["Subject"].isin(subjects_to_include)]
        if df.empty:
            print("No data remaining after subject filtering.")
            return

    # ✅ Group and sort mean/SEM values by agent
    overall_stats = df.groupby("Agent_Type")[metric_name].agg(['mean', 'sem']).reset_index()

    if agent_order:
        # enforce the agent order
        overall_stats["Agent_Type"] = pd.Categorical(overall_stats["Agent_Type"], categories=agent_order, ordered=True)
        overall_stats = overall_stats.sort_values("Agent_Type")

    # ✅ Pivot per subject for line overlays
    try:
        pivot_df = df.pivot(index="Subject", columns="Agent_Type", values=metric_name)
        if agent_order:
            pivot_df = pivot_df[agent_order]  # reorder columns
    except Exception as e:
        print("Error pivoting data:", e)
        return

    # ✅ Paired t-tests
    t_test_results = perform_t_tests(pivot_df)

    fig, ax = plt.subplots(figsize=figsize)
    bar_positions = np.arange(len(overall_stats))

    ax.bar(bar_positions, overall_stats["mean"], yerr=overall_stats["sem"],
           capsize=6, color=bar_color, edgecolor='black', linewidth=5, width=0.6,
           error_kw=dict(elinewidth=3, capthick=3, zorder=5))

    for subject_id in pivot_df.index:
        ax.plot(bar_positions, pivot_df.loc[subject_id], linestyle='-', color='gray', 
                alpha=0.5, linewidth=3, marker='o', markerfacecolor='none', 
                markeredgecolor='gray', markeredgewidth=2, markersize=10)

    ax.set_ylabel(ylabel, fontsize=35, labelpad=12)
    ax.set_xlabel(xlabel, fontsize=35, labelpad=12)
    ax.set_title(title, fontsize=28)

    ax.set_xticks(bar_positions)
    xtick_labels = custom_xtick_labels if custom_xtick_labels else overall_stats["Agent_Type"].tolist()
    ax.set_xticklabels(xtick_labels, fontsize=28)
    if custom_xtick_colors:
        for tick, color in zip(ax.get_xticklabels(), custom_xtick_colors):
            tick.set_color(color)

    ax.tick_params(axis='y', labelsize=35)
    ax.tick_params(axis='x', labelsize=35)

    if ylim:
        ax.set_ylim(ylim)
    else:
        all_vals = np.concatenate([pivot_df.values.flatten(), overall_stats["mean"].values])
        ax.set_ylim(0, np.nanmax(all_vals) * 1.2)

    if yticks_increment:
        y_min, y_max = ax.get_ylim()
        ax.set_yticks(np.arange(np.floor(y_min), np.ceil(y_max) + yticks_increment, yticks_increment))

    ax.axhline(y=0, color='black', linestyle='--', linewidth=2)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(5)
    ax.spines['bottom'].set_linewidth(5)

    plt.tight_layout()
    if save:
        if save_name is None:
            raise ValueError("save_name must be provided if save is True.")
        plt.savefig(save_name, transparent=True, bbox_inches='tight', pad_inches=pad_inches)
    plt.show()

    if t_test_results:
        print("\nPaired t-test results (comparing first agent type to others):")
        for comparison, stats in t_test_results.items():
            print(f"{comparison}: p = {stats['p_value']:.4f} ({stats['significance']})")


plot_da_by_agent_type(
    precomputed_df=da_with_agents_df,
    metric_name="Max Peak",
    title=None,
    ylabel="Peak Z-scored ∆F/F",
    xlabel="Bout Type",
    agent_order=["nothing", "short_term", "long_term", "novel"],
    subjects_to_include= Pref,
    custom_xtick_labels=["Empty", "Short Term", "Long Term", "Novel"],
    custom_xtick_colors=None,
    ylim=(-1, 3),
    yticks_increment=2,
    bar_color='white',
    figsize=(14, 8),
    save=None,
    save_name ="mPFC_C1_3"
)

No data remaining after subject filtering.
